In [ ]:
Для выполнения задания используются датасеты биржевой информации, расположенные по ссылке

sp_data2.csv
sp500hst.csv
Для того, чтобы считать данные файлы в системе Я.Контест, вам нужно использовать относительный путь, как-будто этот файл находится в той же директории, что и исполняемый .py файл, например: pd.read_csv('./sp_data2.csv')

Описание формата данных: Для изучения формата данных, используемых в задании, обратитесь к следующим источникам:

Создание графиков объема и свечных диаграмм
Описание биржевых данных (OHLC)
Wikipedia: Графики OHLC
Перед началом выполнения задания ознакомьтесь с файлами и их структурой.

Задание 1 (базовое):
Загрузка данных
Загрузите данные из файлов в таблицы Pandas с именами:
ticker_data (для файла sp_data2.csv)
ohlc_data (для файла sp500hst.csv)
Обозначьте столбцы для таблицы ticker_data как:

"ticker" (тикер),
"company" (название компании),
"percent" (процентное изменение).
Для таблицы ohlc_data используйте следующие наименования столбцов:

"date" (дата),
"ticker" (тикер),
"volume" (объем),
"open" (цена открытия),
"high" (максимальная цена),
"low" (минимальная цена),
"close" (цена закрытия).
Порядок столбцов должен соответствовать вышеуказанному. Дата в столбце "date" должна быть преобразована в формат datetime.

Создание нового столбца
Добавьте в таблицу ohlc_data новый столбец "mid", который содержит средние значения между дневным минимумом и максимумом. Столбец должен находиться в конце таблицы.
Функция ohlc_agg_data(tickers)
Реализуйте функцию, которая создает DataFrame, индексом которого должны быть тикеры из tickers. В этом DataFrame для каждого тикера необходимо оставить только одну строку, содержащую средние значения по столбцам "volume", "open", "high", "low", "close", "mid" за все даты. В столбце "date" должен храниться первый день, для которого есть данные по каждому тикеру в исходной таблице. Порядок вывода строк тикеров должен быть такой же, как в переданном списке tickers.
Функция vol_diff(ticker1, ticker2, k)
Реализуйте функцию, которая создает DataFrame, содержащий разницу в объемах торгов акциями с тикерами ticker1 и ticker2 в одинаковые даты. Если объем торгов известен только для одного из тикеров, соответствующая строка в результирующую таблицу не добавляется. Параметр k задает количество строк, которые будут выведены из результирующей таблицы, начиная с первой строки. Если k=4, функция должна вернуть строки с индексами от 0 до 4 включительно. Столбцы новой таблицы:
"date" (дата),
"ticker1" (первый тикер),
"ticker2" (второй тикер),
"volume1" (объем торгов для первого тикера),
"volume2" (объем торгов для второго тикера),
"diff" (разница объемов).
Функция max_after_date(date_str)
Реализуйте функцию, которая по данным из sp500hst.csv с помощью Pandas рассчитывает максимальные значения для столбцов 3–6 включительно для дат после 2010-08-01 (дата задается как параметр).
Функция mid_vol_open_gt(open_value)
Реализуйте функцию, которая рассчитывает средний объем торгов для значений "open" больше 50 (значение 50 передается как параметр).
Консоль пользователя
Тут представлена реализация консоли, позволяющую пользователю вводить выражения с использованием Pandas, а результат выполнения выводить в виде CSV-файла. Для этого используется функция eval (подробнее о функции см. здесь). Эту функцию необходимо использовать для ввода команд.

In [ ]:
import pandas as pd

# Загрузка данных
ticker_data = pd.read_csv('./sp_data2.csv', sep=';', names=["ticker", "company", "percent"])

# Округляем столбец percent сразу после преобразования его в float
ticker_data['percent'] = ticker_data['percent'].str.rstrip('%').astype(float).round(2)  # Убираем символ процента и округляем

ohlc_data = pd.read_csv('./sp500hst.csv', names=["date", "ticker", "volume", "open", "high", "low", "close"])

# Преобразуем дату в формат datetime
ohlc_data['date'] = pd.to_datetime(ohlc_data['date'], format='%Y%m%d')

# Округляем числовые столбцы сразу после загрузки
ohlc_data[['volume', 'open', 'high', 'low', 'close']] = ohlc_data[['volume', 'open', 'high', 'low', 'close']].round(2)

# Добавляем новый столбец "mid" - среднее значение между low и high
ohlc_data['mid'] = ((ohlc_data['low'] + ohlc_data['high']) / 2).round(2)


# Функция агрегирования данных по тикерам
def ohlc_agg_data(tickers):
    # Фильтруем данные по тикерам и группируем
    agg_data = ohlc_data[ohlc_data['ticker'].isin(tickers)].groupby('ticker').agg({
        'volume': 'mean',
        'open': 'mean',
        'high': 'mean',
        'low': 'mean',
        'close': 'mean',
        'mid': 'mean',
        'date': 'min'  # минимальная дата для каждого тикера
    })

    # Округляем до двух знаков и возвращаем результат
    return agg_data.round(2).reset_index()


# Функция для расчета разницы в объеме торгов между двумя тикерами
def vol_diff(ticker1, ticker2, k):
    # Фильтруем данные по тикерам
    filtered = ohlc_data[ohlc_data['ticker'].isin([ticker1, ticker2])]

    # Поворотная таблица по объему торгов с тикерами как столбцами
    pivot_table = filtered.pivot_table(index='date', columns='ticker', values='volume', aggfunc='first')

    # Убираем строки, где отсутствуют данные по одному из тикеров
    pivot_table = pivot_table.dropna(subset=[ticker1, ticker2])

    # Вычисляем разницу объемов торгов
    pivot_table['diff'] = (pivot_table[ticker1] - pivot_table[ticker2]).round(2)

    # Переименовываем столбцы для удобства
    pivot_table = pivot_table.rename(columns={ticker1: 'volume1', ticker2: 'volume2'})

    # Формируем результат в виде DataFrame с индексом для вывода
    result = pivot_table.reset_index()[['date', 'volume1', 'volume2', 'diff']].head(k + 1)
    result['ticker1'] = ticker1
    result['ticker2'] = ticker2

    # Добавляем пустой индекс для формата
    result.insert(0, '', range(len(result)))

    return result[['', 'date', 'ticker1', 'ticker2', 'volume1', 'volume2', 'diff']]


# Функция для нахождения максимальных значений после заданной даты
def max_after_date(date_str):
    # Преобразование строки в дату
    date = pd.to_datetime(date_str)

    # Фильтруем данные по дате
    filtered_data = ohlc_data[ohlc_data['date'] > date]

    # Находим максимальные значения по столбцам open, high, low, close
    max_values = (filtered_data[['open', 'high', 'low', 'close']].max()).round(2)

    # Преобразуем данные в DataFrame с нужной структурой
    result = max_values.reset_index().rename(columns={'index': '', 0: 'max_value'}).round(2)
    result.columns = ['', '0']

    return result


# Функция для расчета среднего объема торгов, где open больше указанного значения
def mid_vol_open_gt(open_value):
    # Фильтруем данные по условию
    filtered = ohlc_data[ohlc_data['open'] > open_value]

    # Рассчитываем средний объем торгов
    return filtered['volume'].mean().round(2)


# Основная функция для взаимодействия с пользователем через консоль
def main():
    input_string = eval(input())  # Выполняем ввод пользователя
    if isinstance(input_string, pd.DataFrame) or isinstance(input_string, pd.Series):
        print(input_string.to_csv(index=False))
    else:
        print(input_string)


if __name__ == "__main__":
    main()
